In [1]:
'''MultiHash PCY algorithm from the scratch ---> implementation on finding Asscociation Rules by using pySpark framework '''
import os
import sys
import findspark
findspark.init()
from pyspark import SparkContext,SparkConf,AccumulatorParam
conf=SparkConf()
sc = SparkContext(conf=conf ,master="local[*]")
text = sc.textFile("browsing.txt")
text=text.map(lambda x:x.strip().split(' '))

class MyHash(object):
    def __init__(self,v):
        self.v=v
    def __eq__(self,other):
        return self.v==other.v
    def __hash__(self):
        return hash(self.v)+42

'''subclass the AccumulatroParam helper object and overwrite the addInPlace method'''
class DictAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue):
        return initialValue
    def addInPlace(self, v1, v2):
        for x in v2:
            if x in v1:
                v1[x]+=v2[x]
            else:
                v1[x]=v2[x]
        return v1
'''create the shared dict which is used on each worker node'''    
ac1=sc.accumulator(dict(), DictAccumulatorParam())
ac2=sc.accumulator(dict(), DictAccumulatorParam())
'''hash each item on one bucket from each sample and increment by one the coresponding value '''
def create_2_dicts(it,ac1,ac2):
    d1=dict()
    d2=dict()
    if it in d1.keys():
        d1[it]+=1
    else:
        d1[it]=1
    ob=MyHash(it)
    if ob in d2.keys():
        d2[ob.v]+=1
    else:
        d2[ob.v]=1
    ac1.add(d1)
    ac2.add(d2)
    
def create_pairs(x):
    temp=[]
    for i in range(len(x)):
        for j in range(i+1,len(x)):
            temp.append(tuple(sorted((x[i],x[j]))))
    return temp

count=text.flatMap(lambda x:x).map(lambda x:(x,1)).countByKey()

text=text.map(lambda x:create_pairs(x))

count_d1_d2=text.flatMap(lambda x:x).foreach(lambda x:create_2_dicts(x,ac1,ac2))
d1=ac1.value
d2=ac2.value
#print(max(list(d1.values())),"---->",max(list(d2.values())))
print("Number of frequent singleton items --->",len(count))

def filtering_itemset(x,d1,d2,count):
    temp=[]
    for i in x:
        if i in d1.keys() and i in d2.keys():
            if d1[i]>=100 and d2[i]>=100 and count[i[0]]>=100 and count[i[1]]>=100:
                temp.append(i)
    return temp

text=text.map(lambda x: filtering_itemset(x,d1,d2,count))
text=text.filter(lambda x:x!=[])

'''computer the counts of frequent pairs and compute condidence along with Association rules '''
freq=text.flatMap(lambda x:x).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=100)
print(type(freq))
print("Number of frequent pair items --->",freq.count())

def compute_confidence(x,countIDs):
    temp=[]
    for i in range(len(x[0])):
        temp.append(x[1]/countIDs[x[0][i]])
    idx=temp.index(max(temp))
    ret=()
    for i in range(len(x[0])):
        if i!=idx:
            ret+=(x[0][i],)
    return ret+(x[0][idx],max(temp))
Association_Rule_1=freq.map(lambda x:compute_confidence(x,count))

print("Association Rules for pair frequent Itemsets with Confedence>0.95)",sorted(Association_Rule_1.filter(lambda x:x[2]>0.95).take(8)))

'''Find frequend tripletes of items'''
def create_triples_and_beyond(l):
    temp=[]
    for i in range(len(l)):
        for j in range(i+1,len(l)):
            for z in l[j]:
                if z not in l[i]:
                    t=tuple(sorted(l[i]+(z,)))
                    if t not in temp:
                        temp.append(t)
    return temp

text=text.map(lambda x:create_triples_and_beyond(x))
ac1=sc.accumulator(dict(), DictAccumulatorParam())
ac2=sc.accumulator(dict(), DictAccumulatorParam())

count_d1_d2=text.flatMap(lambda x:x).foreach(lambda x:create_2_dicts(x,ac1,ac2))

d11=ac1.value#sc.broadcast(ac1.value)
d22=ac2.value#sc.broadcast(ac2.value)
count1=freq.collectAsMap()

def filtering_itemset1(x,d1,d2,count):
    temp=[]
    for i in x:
        if i in d1.keys() and i in d2.keys():
            if d1[i]>=100 and d2[i]>=100 and count[i[0]]>=100 and count[i[2]]>=100 and count[i[1]]>=100:
                temp.append(i)
    return temp

text=text.map(lambda x: filtering_itemset1(x,d11,d22,count)).filter(lambda x:x!=[])
print("text--->",text.take(1))
'''computer the counts of frequent triples and compute condidence along with Association rules '''
freq=text.flatMap(lambda x:x).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=100)
print(type(freq))
print("Number of frequent triple items --->",freq.count())
#tempFile = NamedTemporaryFile(delete=True)
#tempFile.close()

#sc.broadcast
Association_Rule_2=freq.map(lambda x:compute_confidence(x,count))
print(type(Association_Rule_2))
print("Association for three pair frequent Itemsets Rule with Conf>0.55)",sorted(Association_Rule_2.filter(lambda x:x[3]>=0.55).take(20)))
Association_Rule_.saveAsTextFile('temp.txt')
sc.stop()

Number of frequent singleton items ---> 12592
<class 'pyspark.rdd.PipelinedRDD'>


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 6, localhost, executor driver): org.apache.spark.SparkException: Python worker did not connect back in time
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:138)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:67)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:128)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:323)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:287)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:87)
	at org.apache.spark.sche